<a href="https://colab.research.google.com/github/i40-Tools/I40KG-Embeddings/blob/master/Similarity_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I40 standards landscape similarity analysis using embeddings

## Overview

In this notebook, we show the similarity analysis between Industry 4.0 Standards. 
To do so, we create embeddings about the Industry 4.0 Standards Knowledge Graph (I40KG) developed by [Grangel-Gonzales et. al.](https://www.researchgate.net/publication/318208930_The_Industry_40_Standards_Landscape_from_a_Semantic_Integration_Perspective)

The embeddings are located here: [I40 Embeddings](https://github.com/i40-Tools/I40KG-Embeddings/tree/master/logs_sto)

## Initial Configurations
First, let's import the required libraries to perform the similarity analysis.

In [0]:
# %ls

I40KG-Embeddings/  sample_data/


In [0]:
# !git clone https://github.com/i40-Tools/I40KG-Embeddings.git

Cloning into 'I40KG-Embeddings'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 85 (delta 36), reused 37 (delta 9), pack-reused 0
Unpacking objects: 100% (85/85), done.


In [0]:
import scipy
from scipy import spatial
import numpy as np
import math
import json
from scipy.spatial.distance import cdist

### Define similarity function
We are going to us cosine distance to measure the similarity between two embeddings

In [0]:
#function to calculate cosine distance 
def cosine_similarity(vec1,vec2):
    sum11, sum12, sum22 = 0, 0, 0
    for i in range(len(vec1)):
        x = vec1[i]; y = vec2[i]
        sum11 += x*x
        sum22 += y*y
        sum12 += x*y
    return sum12/math.sqrt(sum11*sum22)

## Similarity among I40 Standarization Frameworks
In this section we show the analysis of similarity among standarization frameworks

In [0]:
!pip3 install rdflib

     |████████████████████████████████| 348kB 2.8MB/s 
     |████████████████████████████████| 51kB 18.5MB/s 


In [0]:
import json
from rdflib import Graph
import pprint

###Similarity among Frameworks
In this section we analyze the similarity of I4.0 frameworks at the high level, i.e., StandardizationFramework.


In [0]:
g = Graph()
g.parse("/content/I40KG-Embeddings/sto/sto.nt", format="nt")
    
len(g) # prints 2
 #check printing of the graph    
'''for stmt in g:
    pprint.pprint(stmt)'''

#query to get the framework/standard from the sto.nt file
#to get standards we have to replace sto:StandardizationFramework by sto:Standard in the query
#we can get standards of the frameworks as well by just changing the query

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:StandardizationFramework .
            } limit 1000""")
    
       
#to get the corresponding embeddings of the frameworks/standards from the json file 
with open("/content/I40KG-Embeddings/logs_sto/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres:
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
            print(new_dict)

#to put the frameworks/standards with their corresponding embeddings in a file            
with open('/content/I40KG-Embeddings/output.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(new_dict, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f)
    
    
#to read the file containing standards/frameworks along with their embeddings   
with open('/content/I40KG-Embeddings/output.json', 'r') as f:
    array = json.load(f)
    
#compare each standard/framework with all the other standards/frameworks to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards/frameworks to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards/frameworks along with their similar standards/frameworks and their similarity distance
print(result)

{'https://w3id.org/i40/sto#TrustedConnector': [0.1949988156557083, -0.11950734257698059, -0.0136695122346282, -0.21719948947429657, -0.05460599437355995, -0.1717989146709442, 0.14903844892978668, 0.1125185638666153, 0.07403145730495453, -0.22211167216300964, 0.13523592054843903, 0.19295614957809448, -0.13666413724422455, -0.11740493774414062, -0.045872628688812256, -0.07088872790336609, 0.07691168040037155, -0.12950709462165833, 0.0998048260807991, -0.001626146724447608, -0.16794651746749878, -0.00034035733551718295, -0.15628062188625336, 0.09967624396085739, 0.15278302133083344, 0.12049532681703568, -0.14746050536632538, 0.18800075352191925, -0.04131637513637543, -0.14699198305606842, 0.09558244049549103, 0.025179024785757065, 0.10614684969186783, 0.07488871365785599, 0.24369308352470398, 0.00592420669272542, 0.19164210557937622, -0.20703701674938202, -0.17345082759857178, 0.04940425977110863, 0.23710329830646515, 0.15390077233314514, -0.01935317926108837, 0.028283772990107536, 0.1868

####Results
The following table shows the frameworks with high degree of similarites.

| Framework A | Framework B | Score |
| ----------------------|:---------------------:| ----------:|
| RAMI 4.0 |  IIRA  | 0.75 |


In [0]:
# TODO implement a fuction to print a table as the one above

###Similarity among different layers of the Standards
In this section we analyze the similarity among layers of different standards, e.g., RAMI 4.0 -> Layer A vs IIRA -> Layer X